In [1]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
import json
from datetime import datetime, timedelta
from TakeoutParser import TakeoutParser

# set up the directory system
data_directory = str(Path.home()) + "/class/s25/public-health/project3/jackson-data"
garmin_directory = data_directory + "/garmin_data/"
takeout_directory = data_directory + "/takeout/takeout-files/"
timeline_directory = data_directory + "/timeline/"


In [4]:
search = pd.read_json("../../data/Activity/Search/MyActivity.json")
yt = pd.read_json("../../data/Activity/YouTube/MyActivity.json")


In [6]:
search

,header,title,titleUrl,time,products,activityControls,locationInfos,details
0,Search,Searched for google takeout,https://www.google.com/search?q=google+takeout,2025-04-27T22:44:36.364Z,[Search],[Web & App Activity],"[{'name': 'At this general area', 'url': 'http...",NaN
1,Search,Searched for python tqdm,https://www.google.com/search?q=python+tqdm,2025-04-27T22:07:37.367Z,[Search],[Web & App Activity],"[{'name': 'At this general area', 'url': 'http...",NaN
2,Search,Searched for python google generative ai library,https://www.google.com/search?q=python+google+...,2025-04-27T22:07:22.558Z,[Search],[Web & App Activity],"[{'name': 'At this general area', 'url': 'http...",NaN
3,Search,Searched for how to fetch main branch,https://www.google.com/search?q=how+to+fetch+m...,2025-04-27T22:04:50.074Z,[Search],[Web & App Activity],"[{'name': 'At this general area', 'url': 'http...",NaN
4,Search,Searched for TypeError: sample(): incompatible...,https://www.google.com/search?q=TypeError:+sam...,2025-04-27T19:57:33.094Z,[Search],[Web & App Activity],"[{'name': 'At this general area', 'url': 'http...",NaN
...,...,...,...,...,...,...,...,...
15950,Search,"Visited College Board - SAT, AP, College Searc...",https://www.google.com/url?q=https://www.colle...,2018-01-11T21:31:14.629Z,[Search],[Web & App Activity],NaN,NaN
15951,Search,Searched for college board,https://www.google.com/search?q=college+board,2018-01-11T21:30:52.246Z,[Search],[Web & App Activity],NaN,NaN
15952,Search,Visited https://www.google.com/gmail/,https://www.google.com/gmail/,2018-01-11T21:22:04.639Z,[Search],[Web & App Activity],NaN,NaN
15953,Search,Searched for gmail,https://www.google.com/search?q=gmail,2018-01-11T21:22:00.029Z,[Search],[Web & App Activity],NaN,NaN


In [2]:
def ms_to_minutes(ms):
    return ms / 1000 / 60

def extract_first_key(dlist, key):
    if isinstance(dlist, list) and len(dlist) > 0 and isinstance(dlist[0], dict):
        return dlist[0].get(key)
    return None

In [3]:
# load up the Garmin data
sleep = pd.read_json(garmin_directory + "sleep.json")
with open(garmin_directory + "summarizedActivities.json") as f:
        raw_data = json.load(f)[0]['summarizedActivitiesExport']
activities = pd.DataFrame(raw_data)
wellness = pd.read_json(garmin_directory + "/wellness-metrics.json")

# select the features we want

# wellness 
wellness['duration'] = wellness['durationInMilliseconds'].apply(ms_to_minutes)
wellness = wellness[['calendarDate', 'duration', 'totalSteps', 'moderateIntensityMinutes', 'vigorousIntensityMinutes', 'minAvgHeartRate', 'maxAvgHeartRate', 'restingHeartRate',
                      'allDayStress', "bodyBattery"]]
wellness['aggregatorList'] = wellness['allDayStress'].apply(lambda x: x.get('aggregatorList'))
wellness['list0'] = wellness['aggregatorList'].apply(lambda x: x[0] if x else None)
# wellness['maxStressLevel'] = wellness['aggregatorList'].apply(lambda x: x[0]['maxStressLevel'] if x else None)
wellness['averageStressLevel'] = wellness['list0'].apply(lambda x: x.get('averageStressLevel'))
wellness['maxStressLevel'] = wellness['list0'].apply(lambda x: x.get("maxStressLevel"))
wellness = wellness.drop(["aggregatorList", "list0", "allDayStress"], axis=1)

# sleep
sleep = sleep[['calendarDate', "sleepStartTimestampGMT", "sleepEndTimestampGMT", "sleepWindowConfirmationType", "deepSleepSeconds", "lightSleepSeconds"]]


# activity
activities['calendarDate'] = activities['beginTimestamp'].apply(lambda x: datetime.utcfromtimestamp(x / 1000))
activities = activities[['calendarDate', 'activityType', 'name', 'sportType', 'avgHr', 'maxHr', 'calories', 'bmrCalories', 'duration', 'moderateIntensityMinutes', 'vigorousIntensityMinutes']]

# convert to datetime as needed
wellness['calendarDate'] = pd.to_datetime(wellness['calendarDate'])
sleep['calendarDate'] = pd.to_datetime(sleep['calendarDate'])
activities['calendarDate'] = pd.to_datetime(activities['calendarDate'])

wellness['calendarDate'] = wellness['calendarDate'].dt.strftime('%Y-%m-%d')
sleep['calendarDate'] = sleep['calendarDate'].dt.strftime('%Y-%m-%d')
activities['calendarDate'] = activities['calendarDate'].dt.strftime('%Y-%m-%d')

# filter dates
start_date = '2025-01-20'
end_date = '2025-04-21'

wellness = wellness[(wellness['calendarDate'] >= start_date) & (wellness['calendarDate'] <= end_date)]
sleep = sleep[(sleep['calendarDate'] >= start_date) & (sleep['calendarDate'] <= end_date)]
activities = activities[(activities['calendarDate'] >= start_date) & (activities['calendarDate'] <= end_date)]


# joining together the tables on the date

step_one = pd.merge(sleep, activities, on='calendarDate', how='outer')
step_two = pd.merge(step_one, wellness, on='calendarDate', how='outer')

FileNotFoundError: File /Users/jacksonmiskill/class/s25/public-health/project3/jackson-data/garmin_data/sleep.json does not exist

In [40]:
step_one

,calendarDate,sleepStartTimestampGMT,sleepEndTimestampGMT,sleepWindowConfirmationType,deepSleepSeconds,lightSleepSeconds,activityType,name,sportType,avgHr,maxHr,calories,bmrCalories,duration,moderateIntensityMinutes,vigorousIntensityMinutes
0,2025-01-20,2025-01-20T04:00:00.0,2025-01-20T12:00:00.0,UNCONFIRMED,NaN,NaN,running,Charlottesville Carrera,RUNNING,150.0,173.0,2614.57248,272.35130,2.489348e+06,1.0,45.0
1,2025-01-21,2025-01-21T04:00:00.0,2025-01-21T12:00:00.0,UNCONFIRMED,NaN,NaN,strength_training,Strength,TRAINING,101.0,122.0,930.18444,243.02116,2.218154e+06,25.0,0.0
2,2025-01-22,2025-01-22T04:00:00.0,2025-01-22T12:00:00.0,UNCONFIRMED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-01-23,2025-01-23T04:00:00.0,2025-01-23T12:00:00.0,UNCONFIRMED,NaN,NaN,strength_training,Strength,TRAINING,120.0,144.0,2283.56090,435.76208,3.987627e+06,40.0,22.0
4,2025-01-24,2025-01-24T04:00:00.0,2025-01-24T12:00:00.0,UNCONFIRMED,NaN,NaN,strength_training,Strength,TRAINING,132.0,159.0,2270.99084,339.39162,3.118239e+06,15.0,33.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,2025-04-18,2025-04-18T04:09:00.0,2025-04-18T11:11:00.0,ENHANCED_CONFIRMED_FINAL,1080.0,13080.0,lap_swimming,Natación en piscina,SWIMMING,139.0,159.0,2141.10022,331.01158,3.014872e+06,14.0,46.0
105,2025-04-19,2025-04-19T04:48:00.0,2025-04-19T12:35:00.0,ENHANCED_CONFIRMED,480.0,17340.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,2025-04-20,2025-04-20T06:13:00.0,2025-04-20T14:29:00.0,OFF_WRIST,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
107,2025-04-21,2025-04-21T01:35:00.0,2025-04-21T10:24:00.0,ENHANCED_CONFIRMED_FINAL,4080.0,24180.0,indoor_cycling,Ciclismo en sala,CYCLING,142.0,162.0,3163.46510,406.43194,3.693959e+06,6.0,50.0


In [5]:
# reading in timeline data

with open(timeline_directory + "jackson_timeline.json") as f:
    raw_data = json.load(f)

raw_data

timeline = pd.json_normalize(raw_data)
timeline

# merge the timeline data to the other data
timeline['calendarDate'] = pd.to_datetime(timeline['date'])
timeline['calendarDate'] = timeline['calendarDate'].dt.strftime('%Y-%m-%d')

step_three = pd.merge(step_two, timeline, on='calendarDate', how='outer')
step_three


,calendarDate,sleepStartTimestampGMT,sleepEndTimestampGMT,sleepWindowConfirmationType,deepSleepSeconds,lightSleepSeconds,activityType,name,sportType,avgHr,...,restingHeartRate,bodyBattery,averageStressLevel,maxStressLevel,date,person,events.academic,events.work,events.activities,events.notes
0,2025-01-20,2025-01-20T04:00:00.0,2025-01-20T12:00:00.0,UNCONFIRMED,NaN,NaN,running,Charlottesville Carrera,RUNNING,150.0,...,65.0,"{'userProfilePK': 123010558, 'calendarDate': '...",74,90.0,2025-01-20,Jackson,"[{'type': 'class', 'course': 'CS6501-005', 'de...",[],"[{'type': 'laundry', 'description': 'washing, ...",NaN
1,2025-01-21,2025-01-21T04:00:00.0,2025-01-21T12:00:00.0,UNCONFIRMED,NaN,NaN,strength_training,Strength,TRAINING,101.0,...,65.0,"{'userProfilePK': 123010558, 'calendarDate': '...",36,91.0,2025-01-21,Jackson,[],"[{'job_title': 'Data Scientist Inter', 'hours'...",[],NaN
2,2025-01-22,2025-01-22T04:00:00.0,2025-01-22T12:00:00.0,UNCONFIRMED,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,-1,NaN,2025-01-22,Jackson,"[{'type': 'class', 'course': 'CS6501-005', 'de...",[],"[{'type': 'practice music', 'description': 'pr...",
3,2025-01-23,2025-01-23T04:00:00.0,2025-01-23T12:00:00.0,UNCONFIRMED,NaN,NaN,strength_training,Strength,TRAINING,120.0,...,67.0,"{'userProfilePK': 123010558, 'calendarDate': '...",32,83.0,2025-01-23,Jackson,[],"[{'job_title': 'Data Scientist Intern', 'hours...","[{'type': 'Errands', 'description': 'Needed to...",
4,2025-01-24,2025-01-24T04:00:00.0,2025-01-24T12:00:00.0,UNCONFIRMED,NaN,NaN,strength_training,Strength,TRAINING,132.0,...,66.0,"{'userProfilePK': 123010558, 'calendarDate': '...",19,76.0,2025-01-24,Jackson,[],"[{'job_title': 'Data Scientist Intern', 'hours...","[{'type': 'Social', 'description': 'Called my ...",
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,2025-04-18,2025-04-18T04:09:00.0,2025-04-18T11:11:00.0,ENHANCED_CONFIRMED_FINAL,1080.0,13080.0,lap_swimming,Natación en piscina,SWIMMING,139.0,...,56.0,"{'userProfilePK': 123010558, 'calendarDate': '...",44,99.0,2025-04-18,Jackson,"[{'type': 'assignment', 'course': 'CS6501-006'...","[{'job_title': 'Data Scientist Intern', 'hours...","[{'type': 'Concert', 'description': 'Went to a...",
105,2025-04-19,2025-04-19T04:48:00.0,2025-04-19T12:35:00.0,ENHANCED_CONFIRMED,480.0,17340.0,NaN,NaN,NaN,NaN,...,57.0,"{'userProfilePK': 123010558, 'calendarDate': '...",46,99.0,2025-04-19,Jackson,[],[],"[{'type': 'Social', 'description': 'Went to fr...",
106,2025-04-20,2025-04-20T06:13:00.0,2025-04-20T14:29:00.0,OFF_WRIST,0.0,0.0,NaN,NaN,NaN,NaN,...,57.0,"{'userProfilePK': 123010558, 'calendarDate': '...",37,97.0,2025-04-20,Jackson,[],[],"[{'type': 'Social', 'description': 'Had brunch...",
107,2025-04-21,2025-04-21T01:35:00.0,2025-04-21T10:24:00.0,ENHANCED_CONFIRMED_FINAL,4080.0,24180.0,indoor_cycling,Ciclismo en sala,CYCLING,142.0,...,57.0,"{'userProfilePK': 123010558, 'calendarDate': '...",29,93.0,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
# now we need to get the takeout data
# ads, gmail, Image-Search, Maps.html, Search.html, video-search, youtube
directory = takeout_directory
tp = TakeoutParser(directory)
tp.scrape_all_takeout_files()
tp.df

[#######] 100.00%

,date,time,content
0,2025-04-22,18:02:26,Maps
1,2025-04-22,18:00:53,Maps
2,2025-04-22,18:00:07,Maps
3,2025-04-22,18:00:00,Maps
4,2025-04-22,14:02:44,Maps
...,...,...,...
15750,2018-01-11,17:31:14,Search
15751,2018-01-11,17:30:52,Search
15752,2018-01-11,17:22:04,Search
15753,2018-01-11,17:22:00,Search


In [22]:
search = pd.read_json("/Users/jacksonmiskill/class/s25/public-health/project3/Activity/Search/MyActivity.json")
yt = pd.read_json("/Users/jacksonmiskill/class/s25/public-health/project3/Activity/YouTube/MyActivity.json")

full_df = pd.concat([search, yt])

start_date = '2025-01-20'
end_date = '2025-04-21'

full_df = full_df[(full_df['time'] >= start_date) & (full_df['time'] <= end_date)]

final = full_df.reset_index()

final.to_csv("../../jackson-data/final-takeout-data" + ".csv")

In [25]:
tmp_df = tp.df

# joined datetime object
tmp_df['datetime'] = tmp_df.apply(lambda row: datetime.combine(row['date'], row['time']), axis=1)

# shift datetime by 6 hours for the logical day
tmp_df['logical_day'] = (tmp_df['datetime'] - pd.Timedelta(hours=6)).dt.date

result = tmp_df.groupby("logical_day").agg(
    earliest_time=('datetime', lambda x: x.min().time()),
    latest_time=('datetime', lambda x: x.max().time()),
    all_content=('content', list)
).reset_index()

result


,logical_day,earliest_time,latest_time,all_content
0,2018-01-03,23:45:15,23:45:15,[Search]
1,2018-01-11,17:22:00,18:50:08,"[Search, Search, Search, Search, Search, Searc..."
2,2018-01-18,18:42:20,23:40:15,"[Image Search, Image Search, Image Search, Ima..."
3,2018-06-03,08:49:17,08:49:38,"[Search, Search, Search]"
4,2018-09-20,19:21:45,19:21:45,[Search]
...,...,...,...,...
1608,2025-04-18,09:31:47,13:33:57,"[youtube.com, youtube.com, youtube.com, youtub..."
1609,2025-04-19,17:20:13,17:20:13,[youtube.com]
1610,2025-04-21,10:00:41,21:07:06,"[youtube.com, Gmail, Gmail, Gmail, Search, Sea..."
1611,2025-04-22,08:16:43,18:44:12,"[Maps, Maps, Maps, Maps, Maps, YouTube, YouTub..."


In [43]:
result['calendarDate'] = pd.to_datetime(result['logical_day'])
result['calendarDate'] = result['calendarDate'].dt.strftime('%Y-%m-%d')
step_four = pd.merge(step_three, result, on='calendarDate', how='inner')

step_four

,calendarDate,sleepStartTimestampGMT,sleepEndTimestampGMT,sleepWindowConfirmationType,deepSleepSeconds,lightSleepSeconds,activityType,name,sportType,avgHr,...,date,person,events.academic,events.work,events.activities,events.notes,logical_day,earliest_time,latest_time,all_content
0,2025-01-20,2025-01-20T04:00:00.0,2025-01-20T12:00:00.0,UNCONFIRMED,NaN,NaN,running,Charlottesville Carrera,RUNNING,150.0,...,2025-01-20,Jackson,"[{'type': 'class', 'course': 'CS6501-005', 'de...",[],"[{'type': 'laundry', 'description': 'washing, ...",NaN,2025-01-20,11:24:22,23:50:48,"[YouTube, YouTube, YouTube, YouTube, YouTube, ..."
1,2025-01-21,2025-01-21T04:00:00.0,2025-01-21T12:00:00.0,UNCONFIRMED,NaN,NaN,strength_training,Strength,TRAINING,101.0,...,2025-01-21,Jackson,[],"[{'job_title': 'Data Scientist Inter', 'hours'...",[],NaN,2025-01-21,09:01:46,22:34:51,"[YouTube, YouTube, YouTube, YouTube, YouTube, ..."
2,2025-01-22,2025-01-22T04:00:00.0,2025-01-22T12:00:00.0,UNCONFIRMED,NaN,NaN,NaN,NaN,NaN,NaN,...,2025-01-22,Jackson,"[{'type': 'class', 'course': 'CS6501-005', 'de...",[],"[{'type': 'practice music', 'description': 'pr...",,2025-01-22,09:04:15,19:47:29,"[Maps, Maps, Maps, Maps, Maps, Maps, Maps, Map..."
3,2025-01-23,2025-01-23T04:00:00.0,2025-01-23T12:00:00.0,UNCONFIRMED,NaN,NaN,strength_training,Strength,TRAINING,120.0,...,2025-01-23,Jackson,[],"[{'job_title': 'Data Scientist Intern', 'hours...","[{'type': 'Errands', 'description': 'Needed to...",,2025-01-23,09:30:53,01:27:33,"[YouTube, YouTube, youtube.com, youtube.com, y..."
4,2025-01-24,2025-01-24T04:00:00.0,2025-01-24T12:00:00.0,UNCONFIRMED,NaN,NaN,strength_training,Strength,TRAINING,132.0,...,2025-01-24,Jackson,[],"[{'job_title': 'Data Scientist Intern', 'hours...","[{'type': 'Social', 'description': 'Called my ...",,2025-01-24,09:45:38,19:45:26,"[Maps, YouTube, youtube.com, youtube.com, yout..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,2025-04-17,2025-04-17T00:31:00.0,2025-04-17T10:25:00.0,OFF_WRIST,0.0,0.0,strength_training,Strength,TRAINING,118.0,...,2025-04-17,Jackson,"[{'type': 'assignment', 'course': 'CS6501-006'...","[{'job_title': 'Data Scientist Intern', 'hours...","[{'type': 'Errand', 'description': 'Groceries'...",,2025-04-17,08:37:14,13:48:35,"[Maps, Maps, Maps, Maps, Maps, Maps, Maps, Map..."
100,2025-04-18,2025-04-18T04:09:00.0,2025-04-18T11:11:00.0,ENHANCED_CONFIRMED_FINAL,1080.0,13080.0,lap_swimming,Natación en piscina,SWIMMING,139.0,...,2025-04-18,Jackson,"[{'type': 'assignment', 'course': 'CS6501-006'...","[{'job_title': 'Data Scientist Intern', 'hours...","[{'type': 'Concert', 'description': 'Went to a...",,2025-04-18,09:31:47,13:33:57,"[youtube.com, youtube.com, youtube.com, youtub..."
101,2025-04-19,2025-04-19T04:48:00.0,2025-04-19T12:35:00.0,ENHANCED_CONFIRMED,480.0,17340.0,NaN,NaN,NaN,NaN,...,2025-04-19,Jackson,[],[],"[{'type': 'Social', 'description': 'Went to fr...",,2025-04-19,17:20:13,17:20:13,[youtube.com]
102,2025-04-21,2025-04-21T01:35:00.0,2025-04-21T10:24:00.0,ENHANCED_CONFIRMED_FINAL,4080.0,24180.0,indoor_cycling,Ciclismo en sala,CYCLING,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2025-04-21,10:00:41,21:07:06,"[youtube.com, Gmail, Gmail, Gmail, Search, Sea..."


In [44]:
step_four.to_csv(data_directory + ".csv")